In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
import warcraftlogs
from warcraftlogs.constants import TOKEN_DIR

from warcraftlogs import WarcraftLogsClient

client = WarcraftLogsClient(token_dir=TOKEN_DIR)

In [4]:
from warcraftlogs.query.timeline.resources import WarcraftLogsResourceTracker, ResourceType

In [5]:
tracker = WarcraftLogsResourceTracker(client.query_public_api)

In [6]:
PLAYER_NAME = "Jomammadk"

In [8]:
player_data = tracker.track_player_resources("6HXLzJf2PjFdDCn1", 15, PLAYER_NAME )

In [9]:
list_of_playaers = ["Jomammadk", "Shexmage","Intenxity", "Alchyhal","Cellystanis","Dinglesquirt","Buðha", "Amducious", "Hellsangel", "Shungmo", "Sipnsuds"]

In [10]:
list_of_playaers_ids = {player:tracker._find_player_id(report_code="6HXLzJf2PjFdDCn1", character_name=player) for player in list_of_playaers}

In [ ]:
#list_of_playaers = ["Jomammadk"]

In [11]:
players_data = tracker.track_multiple_players(
    "6HXLzJf2PjFdDCn1",
    15,
    list_of_playaers
)

In [12]:
(tracker.fight_end_time- tracker.fight_start_time)

402287

In [13]:
duration = tracker.get_fight_duration_seconds() 
duration

402.287

In [14]:
lock_events = tracker._get_resource_events(report_code="6HXLzJf2PjFdDCn1", fight_id=15, player_id=list_of_playaers_ids["Amducious"])

In [15]:
processed_events = tracker._process_resource_events(lock_events, "lock")

In [ ]:
pd.DataFrame(lock_events)['']

In [ ]:
# increase pandas max columns show to 50
pd.set_option('display.max_columns', 50)

In [23]:
for player_name, data in players_data.items():
    print(f"Player: {player_name}")
    player_resource_types = set()
    for player_snapshot in data:
        player_snapshot_resource = player_snapshot.resources
        for resource_type, resource_amount in player_snapshot_resource.items():
            if resource_type not in player_resource_types:
                player_resource_types.add(resource_type)
                print(f"  Resource Type: {resource_type} name: {ResourceType.from_int(resource_type)}, max amount: {resource_amount.max_amount}")

Player: Jomammadk
  Resource Type: 6 name: ResourceType.RUNIC_POWER, max amount: 1000
  Resource Type: 5 name: ResourceType.RUNES, max amount: 6
Player: Shexmage
  Resource Type: 0 name: ResourceType.MANA, max amount: 3699410
  Resource Type: 11 name: ResourceType.ARCANE_CHARGES, max amount: 4
Player: Intenxity
  Resource Type: 8 name: ResourceType.ASTRAL_POWER, max amount: 1200
Player: Alchyhal
  Resource Type: 2 name: ResourceType.FOCUS, max amount: 100
Player: Cellystanis
  Resource Type: 0 name: ResourceType.MANA, max amount: 2500000
  Resource Type: 9 name: ResourceType.CHI, max amount: 5
Player: Dinglesquirt
  Resource Type: 11 name: ResourceType.ARCANE_CHARGES, max amount: 150
Player: Buðha
  Resource Type: 1 name: ResourceType.RAGE, max amount: 1300
Player: Amducious
  Resource Type: 0 name: ResourceType.MANA, max amount: 2500000
  Resource Type: 7 name: ResourceType.SOUL_SHARDS, max amount: 5
Player: Hellsangel
  Resource Type: 17 name: ResourceType.DEMON_HUNTER_FURY, max amou

In [27]:
players_data['Jomammadk']

[PlayerState(timestamp=25326256, resources={6: ResourceSnapshot(timestamp=25326256, amount=142, max_amount=1000, resource_type=6, change=0, waste=0, is_primary_resource=False), 5: ResourceSnapshot(timestamp=25326256, amount=6, max_amount=6, resource_type=5, change=1, waste=0, is_primary_resource=True)}, hit_points=9288400, max_hit_points=9288400, attack_power=90417, spell_power=9813, armor=64511, item_level=665, position=(-23712, -33214)),
 PlayerState(timestamp=25326256, resources={6: ResourceSnapshot(timestamp=25326256, amount=142, max_amount=1000, resource_type=6, change=5, waste=0, is_primary_resource=True)}, hit_points=9288400, max_hit_points=9288400, attack_power=90417, spell_power=9813, armor=64511, item_level=665, position=(-23712, -33214)),
 PlayerState(timestamp=25327021, resources={6: ResourceSnapshot(timestamp=25327021, amount=342, max_amount=1000, resource_type=6, change=20, waste=0, is_primary_resource=True)}, hit_points=9288400, max_hit_points=9288400, attack_power=11180

In [119]:
player_snapshot

PlayerState(timestamp=25336471, resources={6: ResourceSnapshot(timestamp=25336471, amount=702, max_amount=1000, resource_type=6, change=20, waste=0, is_primary_resource=True)}, hit_points=9288400, max_hit_points=9288400, attack_power=154282, spell_power=9813, armor=64511, item_level=665, position=(-22834, -32586))

In [109]:
players_data['Cellystanis'][20].resources

{0: ResourceSnapshot(timestamp=25345255, amount=2500000, max_amount=2500000, resource_type=0, change=0, waste=0, is_primary_resource=False),
 9: ResourceSnapshot(timestamp=25345255, amount=5, max_amount=5, resource_type=9, change=1, waste=0, is_primary_resource=True)}

In [97]:
{player:len(events)for player, events in players_data.items()}

{'Jomammadk': 534,
 'Shexmage': 363,
 'Intenxity': 248,
 'Alchyhal': 219,
 'Cellystanis': 253,
 'Dinglesquirt': 697,
 'Buðha': 325,
 'Amducious': 113,
 'Hellsangel': 700,
 'Shungmo': 28,
 'Sipnsuds': 222}

In [108]:
players_data['Sipnsuds'][100].resources

{3: ResourceSnapshot(timestamp=25371627, amount=300, max_amount=300, resource_type=3, change=0, waste=0, is_primary_resource=False),
 4: ResourceSnapshot(timestamp=25371627, amount=7, max_amount=7, resource_type=4, change=2, waste=0, is_primary_resource=True)}

In [100]:
(25626779-tracker.fight_start_time)/1000

301.699

In [60]:
runes_30 = tracker.get_resource_at_time(PLAYER_NAME, 30.0, ResourceType.RUNES.value)

Closest state found at 25354610 for Jomammadk at 25355080.0


In [61]:
runes_30

PlayerState(timestamp=25354610, resources={6: ResourceSnapshot(timestamp=25354610, amount=640, max_amount=1000, resource_type=6, change=0, waste=0, is_primary_resource=False)}, hit_points=8593102, max_hit_points=9288400, attack_power=127820, spell_power=9813, armor=64511, item_level=665, position=(-23028, -32818))